In [1]:
#import packages and create date

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
import difflib
from pulp import *
import openpyxl
import glob

from datetime import *
today = date.today()
today = today.strftime("%m.%d.20%y")
today

time = "8pm_"
# import datetime
# today = date.today()
# today = today - datetime.timedelta(days=1)
# today = today.strftime("%m.%d.20%y")
# today

In [3]:
## FanDuel Player Pool

contest = "SingleGame_"
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
contest_data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool'

# fd = glob.glob('*-players-list.csv')[0]

fd_upload = glob.glob('*upload-template.csv')[0]
df = pd.read_csv(fd_upload, header=6, usecols=[10,11,12,13,14,15,16,17,18,19,20,21])
df.to_csv("FDSalaries_"+ contest + time + str(today) + ".csv", index=False)
# os.rename(fd, "FDSalaries_"+ contest + time + str(today) + ".csv")

shutil.move(downloads + "/"+ "FDSalaries_"+ contest + time + str(today) + ".csv",
           contest_data_loc + "/"+ "FDSalaries_"+ contest + time + str(today) + ".csv")

os.chdir(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool")
fanduel_file = pd.read_csv(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool/FDSalaries_"+ contest + time + str(today) + ".csv")
fanduel_file.to_excel(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/FD_Data/PlayerPool/FDSalaries_"+ contest + time + str(today) + ".xlsx", index=False)
os.remove("FDSalaries_"+ contest + time + str(today) + ".csv")

## Uploads
os.chdir(r"C:\Users\vchang\Downloads")

fanduel_upload = r"C:\Users\vchang\Documents\Fantasy Bball\DFS\Upload\FanDuel\Template"

fd_upload = glob.glob('*upload-template.csv')[0]
os.rename(fd_upload, "FD_Upload_"+ contest + time + str(today) + ".csv")

shutil.move(downloads + "/"+ "FD_Upload_"+ contest + time + str(today) + ".csv",
           fanduel_upload + "/"+ "FD_Upload_"+ contest + time + str(today) + ".csv")

'C:\\Users\\vchang\\Documents\\Fantasy Bball\\DFS\\Upload\\FanDuel\\Template/FD_Upload_SingleGame_8pm_06.04.2022.csv'

In [4]:
## ETR Daily

# changes the working directory to your downloads folder
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/ETR_Data/FanDuel'
daily_file = "FD NBA Projections.csv"

shutil.move(downloads + "/"+ daily_file,
           data_loc + "/"+ daily_file)

os.chdir(data_loc)

df = pd.read_csv(daily_file)

df.to_excel("ETR_Daily_"+ time + str(today) +".xlsx", index=False)
os.remove("FD NBA Projections.csv")

In [8]:
## Joins the FD and ETR Files to include FD Positions 
    ## ETR does not include G/F/UTIL

os.chdir(r"C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis")
etr = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\ETR_Data\FanDuel\ETR_Daily_' + time + str(today) + ".xlsx")
fd = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\FD_Data\PlayerPool\FDSalaries_' + contest + time + str(today) + ".xlsx")

df = pd.merge(etr, 
              fd[['Nickname','Position', 'Id', 'Salary']], 
              left_on='Player', 
              right_on='Nickname')

df.head()

,Player,Team,Opp,Minutes,FD Position,FD Salary,FD Points,FD Value,FDCeiling,FD Own,FD Slate,Nickname,Position,Id,Salary
0,Draymond Green,GSW,BOS,33.5,PF,"11,000",31.1,-17.9,40.9,71.8,MAIN,Draymond Green,PF,76966-15860,12500
1,Klay Thompson,GSW,BOS,37.0,SG,"11,500",31.1,-19.9,38.9,85.4,MAIN,Klay Thompson,SG,76966-14509,11500
2,Marcus Smart,BOS,GSW,38.0,SG,"12,000",33.0,-20.0,42.6,79.0,MAIN,Marcus Smart,SG,76966-23670,12000
3,Grant Williams,BOS,GSW,29.0,PF,"8,000",16.9,-20.1,24.8,29.2,MAIN,Grant Williams,PF,76966-110318,8000
4,Jaylen Brown,BOS,GSW,40.0,SF,"14,000",40.4,-20.6,49.7,78.6,MAIN,Jaylen Brown,SF,76966-67026,14000


In [49]:
## Joins the FD and ETR Files to include FD Positions 
    ## ETR does not include G/F/UTIL

os.chdir(r"C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis")
etr = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\ETR_Data\FanDuel\ETR_Daily_' + time + str(today) + ".xlsx")
fd = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\FD_Data\PlayerPool\FDSalaries_' + contest + time + str(today) + ".xlsx")


## TO DO - check if merge on only DKNG Roster Position Works
df = pd.merge(etr, 
              fd[['Nickname','Position', 'Id', 'Salary']], 
              left_on='Player', 
              right_on='Nickname')


# Add 'G' Column
# Add 'F' Column
df['G Position'] = np.where(df['FD Position'].str.contains("G"), 'G', "")
df['F Position'] = np.where(df['FD Position'].str.contains("F"), 'F', "")

df['G Position'] = df['G Position'].replace('', np.nan, regex=True)
df['F Position'] = df['F Position'].replace('', np.nan, regex=True)

# Concat the list of positions
df['list'] = df[['FD Position', 'G Position', 'F Position']].apply(lambda x: '/'.join(x.dropna()), axis=1)

# ## Update Salary to INT 
# df['FD Salary'] = df['FD Salary'].str.replace(",","").astype(float)

df['Name + ID'] = df['Id'] + ":" + df['Player']

df.to_csv("FD_final_SingleGame_ETR_"+ time + str(today) +".csv", index=False)

df = pd.read_csv(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis\FD_final_SingleGame_ETR_'+ time + str(today) + ".csv")


## Unnests FD Positions into each Row (explode in Python) ##

# Update Salary to INT 
# df['DK Salary'] = df['DK Salary'].str.replace(",","").astype(float)

# Update FD Position to a string
df['FD Position'] = df['FD Position'].astype(str)
#Delimit by '/'
df['FD Position'] = df['FD Position'].str.split('/')

# Unnest columns by FD Position
df = df.explode('FD Position')

# Create Position + ID as a lookup value
df['Position + Id'] = df['FD Position'] + "_" + df['Id']

df.head()

df.to_csv("FD_final_SingleGame_ETR_"+ time + str(today) +".csv", index=False)
# os.system("DKNG_final_PlayerPool_ETR_"+ time + str(today) +".xlsx")

# Create the Constraint Problem

Goal: Maximize FD Points

- Total Players = 9
- TotalSalary <= 60000
- TotalPosition_PG = 2
- TotalPosition_SG = 2
- TotalPosition_SF = 2
- TotalPosition_PF =2 
- TotalPosition_C =1


In [50]:
## BASE Optimizer

df = pd.read_csv("FD_final_SingleGame_ETR_"+ time + str(today) +".csv", index_col=['Player', 'FD Position'], skipinitialspace=True)

legal_assignments = df.index # tuples of (name, pos)
name_set = df.index.unique(0) # a convenience

costs = df['Salary'].to_dict()
values = df['FD Points'].to_dict()

# set up LP
draft = pulp.LpVariable.dicts('selected', legal_assignments, cat='Binary')

prob = pulp.LpProblem('the draft', LpMaximize)

#obj
prob += pulp.lpSum([draft[n]*values[n] for (n) in legal_assignments])

#salary cap
prob += pulp.lpSum([draft[n]*costs[n] for (n) in legal_assignments]) <= 60000

# positions
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments]) == 5
# prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'UTIL']) == 5
# prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'SF']) == 2
# prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'PF']) == 2
# prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'C']) == 1

# use each player at most only once
result = []
for name in name_set:
    prob += pulp.lpSum([draft[n] for (n) in legal_assignments if n == name]) <= 1

prob.solve()

for idx in draft:
    if draft[idx].varValue > 0:
        print(idx[0])
        
        result.append([idx[0]])
        
bloop = pd.DataFrame(result, columns = ['Player'])

Klay Thompson
Marcus Smart
Jayson Tatum
Kevon Looney
Andrew Wiggins


C:\Users\vchang\Anaconda3\lib\site-packages\pulp\pulp.py:1313: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [52]:
df = pd.read_csv("FD_final_SingleGame_ETR_"+ time + str(today) +".csv", index_col=['Player', 'FD Position'], skipinitialspace=True)

lineup = pd.merge(bloop, 
              df[['Nickname', 'FD Salary', 'FD Points']], 
              left_on='Player', 
              right_on='Nickname')


lineup = lineup.sort_values(by=['FD Points'], ascending=False)

os.chdir(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\Upload\FanDuel')
lineup.to_csv("FD_Final_Upload_Base_" + contest + time + str(today) +".csv", index=False)

lineup

,Player,Nickname,FD Salary,FD Points
2,Jayson Tatum,Jayson Tatum,"16,000",47.7
1,Marcus Smart,Marcus Smart,"12,000",33.0
0,Klay Thompson,Klay Thompson,"11,500",31.1
4,Andrew Wiggins,Andrew Wiggins,"13,000",30.3
3,Kevon Looney,Kevon Looney,"10,000",22.9
